In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
from collections import OrderedDict
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel("data.xlsx", usecols = [0,1,2,3,4] ,index_col = [0,1,2])
data.head()

heard_pitch answered_pitch
ID Instrument timbre                           
1  piano      sin             c4              c
              sin            g#5              a
              sin             f4              f
              sin             a5              b
              sin            d#3              e

In [3]:
form_data = pd.read_excel("form_answers.xlsx", sheet_name = 'formated', index_col = 0)
form_data.head()

,instr_principal,instr_sec,age,estimated_cap,age_ap,family_ap,age_music_studies,rate_theory,rate_technique,rate_hear,genres_play,genres_listen,habits_band,habits_scores
id,,,,,,,,,,,,,,
1,piano,NaN,18,AP,8.0,No,4,8.0,5.0,8.0,jazz,"jazz, rock, house","playing solo, playing in an orchestra",improvising
2,saxophone,NaN,18,RP,NaN,No,6,7.0,6.0,5.0,jazz,jazz,"playing solo, playing in a small ensemble",improvising
3,trombone,NaN,19,RP,NaN,No,4,6.0,9.0,7.0,"classic, jazz","classic, jazz","playing solo, playing in a band, playing in a ...","reading scores, playing tunes by hearing"
4,trombone,piano,26,RP,NaN,No,6,9.0,7.0,9.0,"classical, jazz","classical, jazz","playing in a band, playing in a small ensemble...","reading scores, improvising, playing tunes by ..."
5,saxophone,NaN,25,RP,NaN,No,6,7.0,6.0,3.0,"jazz, funk, rock","jazz, funk, rock, classic, pop-rock","playing in a small ensemble, playing in an orc...",reading scores


In [4]:
data.dropna()
#data = data[~(str(data['heard_pitch']) == 'nan') & ~(str(data['heard_pitch']) == 'NaN')]
data['heard_pitch'] = data['heard_pitch'].str.replace('[0-9 ]+', "")
data.head()

heard_pitch answered_pitch
ID Instrument timbre                           
1  piano      sin              c              c
              sin             g#              a
              sin              f              f
              sin              a              b
              sin             d#              e

In [5]:
pitch_dict = {
    'c':1, 'c#':2, 'db':2, 'd':3, 'd#':4, 'eb':4, 'e':5, 'fb':5, 'f':6, 'e#':6, 'f#':7, 'gb':7, 'g':8, 'g#':9, 'ab':9, 'a':10, 'a#':11, 'bb':11, 'b':12, 'cb':12, 'b#':1}

def pitch_to_int(x):
    if x in pitch_dict.keys():
        return pitch_dict[x]
    else:
        #print(x)
        return np.NaN

dataT = data.copy()
dataT['heard_pitch'] = dataT['heard_pitch'].apply(lambda x : pitch_to_int(x))
dataT['answered_pitch'] = dataT['answered_pitch'].apply(lambda x : pitch_to_int(x))



#dataT[dataT['heard_pitch'] == np.NaN]

In [6]:
dataT['diff'] = dataT['heard_pitch'] - dataT['answered_pitch']
dataT['abs_diff'] = abs(dataT['heard_pitch'] - dataT['answered_pitch'])
dataT.head()

heard_pitch  answered_pitch  diff  abs_diff
ID Instrument timbre                                             
1  piano      sin             1.0             1.0   0.0       0.0
              sin             9.0            10.0  -1.0       1.0
              sin             6.0             6.0   0.0       0.0
              sin            10.0            12.0  -2.0       2.0
              sin             4.0             5.0  -1.0       1.0

### Statistics

In [7]:
# mean global error
dataT['abs_diff'].mean()
dataT['diff'].mean()

0.29596412556053814

In [8]:
dataT['abs_diff'].groupby('ID').describe()

,count,mean,std,min,25%,50%,75%,max
ID,,,,,,,,
1,36.0,0.583333,0.874234,0.0,0.00,0.0,1.00,4.0
2,35.0,3.628571,3.078251,0.0,1.00,3.0,6.00,10.0
3,35.0,4.400000,2.952566,0.0,2.00,4.0,7.00,10.0
4,34.0,3.235294,3.339922,0.0,1.00,2.0,4.00,11.0
5,34.0,3.911765,2.700927,0.0,2.00,3.5,6.00,9.0
6,36.0,3.916667,3.289594,0.0,1.00,4.0,7.00,11.0
7,36.0,4.194444,3.031449,0.0,1.75,4.0,7.00,11.0
8,36.0,4.833333,3.028437,1.0,2.00,4.0,7.00,11.0
9,36.0,3.277778,2.700382,0.0,1.00,3.5,5.25,8.0


In [10]:
dataT['abs_diff'].groupby('Instrument').describe()


,count,mean,std,min,25%,50%,75%,max
Instrument,,,,,,,,
drums,36.0,0.138889,0.542627,0.0,0.00,0.0,0.0,3.0
guitar,22.0,3.409091,3.142021,0.0,1.00,2.5,5.0,11.0
piano,72.0,1.930556,2.410811,0.0,0.00,1.0,4.0,8.0
saxophone,105.0,3.914286,2.925729,0.0,2.00,3.0,6.0,11.0
trombone,105.0,3.857143,3.202849,0.0,1.00,3.0,7.0,11.0
trumpet,72.0,4.583333,3.033847,0.0,2.00,4.0,7.0,11.0
violin,34.0,3.558824,2.830475,0.0,1.25,3.0,6.0,11.0
